In [ ]:
pip install striprtf

In [ ]:
from striprtf.striprtf import rtf_to_text
with open("/content/algoparams_from_ui.json.rtf", "r") as file:
    rtf_content = file.read()

plain_text = rtf_to_text(rtf_content)

with open("data.json", "w") as json_file:
    json_file.write(plain_text)

print("Conversion complete: 'data.json'")


Conversion complete: 'data.json'


In [ ]:
!pip install pyspark
!pip install findspark
!pip install pandas
!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:

df = pd.read_csv('/content/iris.csv')
df

ohe = OneHotEncoder()
df1 = pd.DataFrame(ohe.fit_transform(df[['species']]).toarray(), columns=df['species'].unique())
df = pd.concat([df, df1], axis=1)
df.drop('species', axis=1, inplace=True)
df

,sepal_length,sepal_width,petal_length,petal_width,Iris-setosa,Iris-versicolor,Iris-virginica
0,5.1,3.5,1.4,0.2,1.0,0.0,0.0
1,4.9,3.0,1.4,0.2,1.0,0.0,0.0
2,4.7,3.2,1.3,0.2,1.0,0.0,0.0
3,4.6,3.1,1.5,0.2,1.0,0.0,0.0
4,5.0,3.6,1.4,0.2,1.0,0.0,0.0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.0,1.0
146,6.3,2.5,5.0,1.9,0.0,0.0,1.0
147,6.5,3.0,5.2,2.0,0.0,0.0,1.0
148,6.2,3.4,5.4,2.3,0.0,0.0,1.0


In [ ]:
df_json = pd.read_json('/content/data.json')
df_json

,session_name,session_description,design_state_data
session_info,test,test,"{'project_id': '1', 'experiment_id': 'kkkk-11'..."
target,test,test,"{'prediction_type': 'Regression', 'target': 'p..."
train,test,test,"{'policy': 'Split the dataset', 'time_variable..."
metrics,test,test,"{'optomize_model_hyperparameters_for': 'AUC', ..."
feature_handling,test,test,{'sepal_length': {'feature_name': 'sepal_lengt...
feature_generation,test,test,"{'linear_interactions': [['petal_length', 'sep..."
feature_reduction,test,test,"{'feature_reduction_method': 'Tree-based', 'nu..."
hyperparameters,test,test,"{'stratergy': 'Grid Search', 'shuffle_grid': T..."
weighting_stratergy,test,test,{'weighting_stratergy_method': 'Sample weights...
probability_calibration,test,test,{'probability_calibration_method': 'Sigmoid - ...


In [ ]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Dendrite').getOrCreate()
spark

In [ ]:
df_json_pyspark = spark.read.option("multiline", "true").json("/content/data.json")
df_json_pyspark.show()
df_json_pyspark.printSchema()

+--------------------+-------------------+------------+
|   design_state_data|session_description|session_name|
+--------------------+-------------------+------------+
|{{{false, 7, 4, [...|               test|        test|
+--------------------+-------------------+------------+

root
 |-- design_state_data: struct (nullable = true)
 |    |-- algorithms: struct (nullable = true)
 |    |    |-- DecisionTreeClassifier: struct (nullable = true)
 |    |    |    |-- is_selected: boolean (nullable = true)
 |    |    |    |-- max_depth: long (nullable = true)
 |    |    |    |-- min_depth: long (nullable = true)
 |    |    |    |-- min_samples_per_leaf: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- model_name: string (nullable = true)
 |    |    |    |-- use_best: boolean (nullable = true)
 |    |    |    |-- use_entropy: boolean (nullable = true)
 |    |    |    |-- use_gini: boolean (nullable = true)
 |    |    |    |-- use_random: 

In [ ]:

from pyspark.sql.functions import col, explode_outer
from pyspark.sql.types import *

def flatten(df):
   complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if isinstance(field.dataType, (ArrayType, StructType))])
   while complex_fields:
      col_name = list(complex_fields.keys())[0]
      print(f"Processing: {col_name} Type: {type(complex_fields[col_name])}")
      if isinstance(complex_fields[col_name], StructType):
         expanded = [col(f"{col_name}.{k}").alias(f"{col_name}_{k}") for k in [n.name for n in complex_fields[col_name]]]
         df = df.select("*", *expanded).drop(col_name)

      elif isinstance(complex_fields[col_name], ArrayType):
         df = df.withColumn(col_name, explode_outer(col_name))

      #
      complex_fields = dict([(field.name, field.dataType)
                             for field in df.schema.fields
                             if isinstance(field.dataType, (ArrayType, StructType))])
   return df

# Flat
df_flatten = flatten(df_json_pyspark)
df_flatten.show()

Processing: design_state_data Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_algorithms Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_feature_generation Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_feature_handling Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_feature_reduction Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_hyperparameters Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_metrics Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_probability_calibration Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_session_info Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_target Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_train Type: <class 'pyspark.sql.types.StructType'>
Processing: design_state_data_weight

Q.1 Read the target and type of regression to be run

In [ ]:
target = df_json.loc['target', 'design_state_data']['target']
type_of_regression = df_json.loc['target', 'design_state_data']['type']

print(f"Target: {target}")
print(f"Type of Regression: {type_of_regression}")

Target: petal_width
Type of Regression: regression


In [ ]:
p = df[["sepal_length", "sepal_width", "petal_length"]]
q = df["petal_width"]

p_train, p_test, q_train, q_test = train_test_split(p, q, test_size=0.2, random_state=42)

models = {
    'Random Forest Regressor': RandomForestRegressor(),
    'GBT Regressor': GradientBoostingRegressor(),
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Elastic Net Regression': ElasticNet(),
    'Decision Tree Regressor': DecisionTreeRegressor()
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(p_train, q_train)
    score = model.score(p_test, q_test)
    print(f'{name}: {score:.3f}')

Random Forest Regressor: 0.936
GBT Regressor: 0.925
Linear Regression: 0.927
Ridge Regression: 0.928
Lasso Regression: 0.329
Elastic Net Regression: 0.698
Decision Tree Regressor: 0.883


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer

In [ ]:

df = pd.read_csv('/content/iris.csv')

#
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
# Load JSON con
df_json = pd.read_json('/content/data.json')

# Extract fea
feature_dict = df_json.loc['feature_handling', 'design_state_data']
feature_dict

{'sepal_length': {'feature_name': 'sepal_length',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': 'Average of values',
   'impute_value': 0}},
 'sepal_width': {'feature_name': 'sepal_width',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': 'custom',
   'impute_value': -1}},
 'petal_length': {'feature_name': 'petal_length',
  'is_selected': True,
  'feature_variable_type': 'numerical',
  'feature_details': {'numerical_handling': 'Keep as regular numerical feature',
   'rescaling': 'No rescaling',
   'make_derived_feats': False,
   'missing_values': 'Impute',
   'impute_with': '

In [ ]:
#
def feature_handling(feature_handling, column_names, df):
    for col in column_names:
        try:
            if feature_handling[col]['feature_details']['impute_with'] == 'custom':
                df[col] = df[col].fillna(feature_handling[col]['feature_details']['impute_value'])
            elif feature_handling[col]['feature_details']['impute_with'] == 'Average of values':
                df[col] = df[col].fillna(df[col].mean())
        except KeyError:
            print(f"Feature {col} is not in the configuration.")
    return df

#
df_imputed = feature_handling(feature_dict, df.columns, df)

#
df_imputed

Feature species is not in the configuration.


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:

spark = SparkSession.builder.appName('Dendrite').getOrCreate()

df_csv_pyspark = spark.read.csv('/content/iris.csv', header=True, inferSchema=True)

df_csv_pyspark.printSchema()

df_csv_pyspark.show()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- species: string (nullable = true)

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|    species|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9

In [ ]:

df_csv_pyspark = df_csv_pyspark.drop('species')

df_csv_pyspark.show()

+------------+-----------+------------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|
+------------+-----------+------------+-----------+
|         5.1|        3.5|         1.4|        0.2|
|         4.9|        3.0|         1.4|        0.2|
|         4.7|        3.2|         1.3|        0.2|
|         4.6|        3.1|         1.5|        0.2|
|         5.0|        3.6|         1.4|        0.2|
|         5.4|        3.9|         1.7|        0.4|
|         4.6|        3.4|         1.4|        0.3|
|         5.0|        3.4|         1.5|        0.2|
|         4.4|        2.9|         1.4|        0.2|
|         4.9|        3.1|         1.5|        0.1|
|         5.4|        3.7|         1.5|        0.2|
|         4.8|        3.4|         1.6|        0.2|
|         4.8|        3.0|         1.4|        0.1|
|         4.3|        3.0|         1.1|        0.1|
|         5.8|        4.0|         1.2|        0.2|
|         5.7|        4.4|         1.5|        0.4|
|         5.

In [ ]:

imputer = Imputer(
    inputCols=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'],
    outputCols=[f"{c}_imputed" for c in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
).setStrategy("mean")


df_imputed_pyspark = imputer.fit(df_csv_pyspark).transform(df_csv_pyspark)


df_imputed_pyspark.show()

+------------+-----------+------------+-----------+--------------------+-------------------+--------------------+-------------------+
|sepal_length|sepal_width|petal_length|petal_width|sepal_length_imputed|sepal_width_imputed|petal_length_imputed|petal_width_imputed|
+------------+-----------+------------+-----------+--------------------+-------------------+--------------------+-------------------+
|         5.1|        3.5|         1.4|        0.2|                 5.1|                3.5|                 1.4|                0.2|
|         4.9|        3.0|         1.4|        0.2|                 4.9|                3.0|                 1.4|                0.2|
|         4.7|        3.2|         1.3|        0.2|                 4.7|                3.2|                 1.3|                0.2|
|         4.6|        3.1|         1.5|        0.2|                 4.6|                3.1|                 1.5|                0.2|
|         5.0|        3.6|         1.4|        0.2|           

In [ ]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np

def apply_feature_reduction(df, target, feature_reduction_config):

    method = feature_reduction_config['feature_reduction_method']

    if method == 'No Reduction':
        num_features_to_keep = feature_reduction_config['No Reduction']['num_of_features_to_keep']
        selected_features = df.columns[:num_features_to_keep]
        df_reduced = df[selected_features]

    elif method == 'Correlation with target':
        if feature_reduction_config['Correlation with target']['is_selected']:
            num_features_to_keep = feature_reduction_config['Correlation with target']['num_of_features_to_keep']
            correlation_matrix = df.corr()
            target_corr = correlation_matrix[target].abs().sort_values(ascending=False)
            selected_features = target_corr.index[1:num_features_to_keep + 1]
            df_reduced = df[selected_features]

    elif method == 'Tree-based':
        if feature_reduction_config['Tree-based']['is_selected']:
            num_features_to_keep = feature_reduction_config['Tree-based']['num_of_features_to_keep']
            depth = feature_reduction_config['Tree-based']['depth_of_trees']
            num_trees = feature_reduction_config['Tree-based']['num_of_trees']
            model = RandomForestClassifier(n_estimators=num_trees, max_depth=depth, random_state=42)
            model.fit(df.drop(columns=[target]), df[target])
            feature_importances = pd.Series(model.feature_importances_, index=df.drop(columns=[target]).columns)
            selected_features = feature_importances.sort_values(ascending=False).head(num_features_to_keep).index
            df_reduced = df[selected_features]

    elif method == 'Principal Component Analysis':
        if feature_reduction_config['Principal Component Analysis']['is_selected']:
            num_features_to_keep = feature_reduction_config['Principal Component Analysis']['num_of_features_to_keep']
            pca = PCA(n_components=num_features_to_keep)
            pca_result = pca.fit_transform(df.drop(columns=[target]))
            df_reduced = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(num_features_to_keep)])

    return df_reduced


In [ ]:

!pip install pyspark
!pip install findspark
!pip install pandas
!pip install scikit-learn
!pip install seaborn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession


In [ ]:

df = pd.read_csv('/content/iris.csv')

df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:

df_json = pd.read_json('/content/data.json')

feature_reduction_config = df_json.loc['feature_reduction', 'design_state_data']
feature_reduction_config


{'feature_reduction_method': 'Tree-based',
 'num_of_features_to_keep': '4',
 'num_of_trees': '5',
 'depth_of_trees': '6'}

In [ ]:


def no_reduction(X, y):
    return X

def corr_with_target(X, y, threshold=0.5):
    corr_with_target = X.corrwith(y).abs()
    features_to_keep = corr_with_target[corr_with_target >= threshold].index
    return X[features_to_keep]

def tree_based(X, y, n_features=3):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X, y)
    feature_importances = model.feature_importances_
    features_to_keep = X.columns[np.argsort(feature_importances)[::-1][:n_features]]
    return X[features_to_keep]

def pca_reduction(X, y, n_components=2):
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    cols = ['PC'+str(i) for i in range(1, n_components+1)]
    X_reduced_df = pd.DataFrame(X_reduced, columns=cols, index=X.index)
    return X_reduced_df

In [ ]:

reduction_methods = {
    'No Reduction': no_reduction,
    'Corr with Target': corr_with_target,
    'Tree-based': tree_based,
    'PCA': pca_reduction
}

In [ ]:
import plotly.graph_objects as go

if selected_method == 'Corr with Target':

    df['species'] = df['species'].astype('category').cat.codes

    # Compute the correlation matrix
    corr_matrix = df.corr()

    # Create a heatmap using plotly
    fig = go.Figure(data=go.Heatmap(
                    z=corr_matrix.values,
                    x=corr_matrix.columns,
                    y=corr_matrix.columns,
                    colorscale='RdBu',
                    zmin=-1, zmax=1,
                    colorbar=dict(title='Correlation', tickvals=[-1, 0, 1])
                ))

    fig.update_layout(
        title="Correlation Matrix",
        xaxis_title="Features",
        yaxis_title="Features",
        xaxis=dict(tickmode='array', tickvals=list(range(len(corr_matrix.columns))), ticktext=corr_matrix.columns),
        yaxis=dict(tickmode='array', tickvals=list(range(len(corr_matrix.columns))), ticktext=corr_matrix.columns)
    )

    fig.show()


In [ ]:
df_json.loc['algorithms']['design_state_data']

{'RandomForestClassifier': {'model_name': 'Random Forest Classifier',
  'is_selected': False,
  'min_trees': 10,
  'max_trees': 30,
  'feature_sampling_statergy': 'Default',
  'min_depth': 20,
  'max_depth': 30,
  'min_samples_per_leaf_min_value': 5,
  'min_samples_per_leaf_max_value': 50,
  'parallelism': 0},
 'RandomForestRegressor': {'model_name': 'Random Forest Regressor',
  'is_selected': True,
  'min_trees': 10,
  'max_trees': 20,
  'feature_sampling_statergy': 'Default',
  'min_depth': 20,
  'max_depth': 25,
  'min_samples_per_leaf_min_value': 5,
  'min_samples_per_leaf_max_value': 10,
  'parallelism': 0},
 'GBTClassifier': {'model_name': 'Gradient Boosted Trees',
  'is_selected': False,
  'num_of_BoostingStages': [67, 89],
  'feature_sampling_statergy': 'Fixed number',
  'learningRate': [],
  'use_deviance': True,
  'use_exponential': False,
  'fixed_number': 22,
  'min_subsample': 1,
  'max_subsample': 2,
  'min_stepsize': 0.1,
  'max_stepsize': 0.5,
  'min_iter': 20,
  'max_i

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
p_train, p_test, q_train, q_test = train_test_split(p, q, test_size=0.2, random_state=42)

In [ ]:

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(p_train, q_train)
rf_pred = rf.predict(p_test)
print('Random Forest Regressor: ', mean_squared_error(q_test, rf_pred))


gbt = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gbt.fit(p_train, q_train)
gbt_pred = gbt.predict(p_test)
print('GBTRegressor: ', mean_squared_error(q_test, gbt_pred))


lr = LinearRegression()
lr.fit(p_train, q_train)
lr_pred = lr.predict(p_test)
print('Linear Regression: ', mean_squared_error(q_test, lr_pred))

ridge = Ridge(alpha=1.0)
ridge.fit(p_train, q_train)
ridge_pred = ridge.predict(p_test)
print('Ridge Regression: ', mean_squared_error(q_test, ridge_pred))


lasso = Lasso(alpha=0.1)
lasso.fit(p_train, q_train)
lasso_pred = lasso.predict(p_test)
print('Lasso Regression: ', mean_squared_error(q_test, lasso_pred))


enet = ElasticNet(alpha=0.1, l1_ratio=0.5)
enet.fit(p_train, q_train)
enet_pred = enet.predict(p_test)
print('Elastic Net Regression: ', mean_squared_error(q_test, enet_pred))

dt = DecisionTreeRegressor()
dt.fit(p_train, q_train)
dt_pred = dt.predict(p_test)
print('Decision Tree Regressor: ', mean_squared_error(q_test, dt_pred))

Random Forest Regressor:  0.04426238323299315
GBTRegressor:  0.05377212033251875
Linear Regression:  0.04643810173004898
Ridge Regression:  0.04545986663436361
Lasso Regression:  0.051572308970345065
Elastic Net Regression:  0.04899187824212669
Decision Tree Regressor:  0.06933333333333332


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


p_train, p_test, q_train, q_test = train_test_split(p, q, test_size=0.2, random_state=42)

#
models_config = {
    "RandomForestRegressor": {"model": RandomForestRegressor(),
                              "params": {"n_estimators": [50, 100, 200],
                                         "max_features": ["sqrt", "log2"]}},
    "GradientBoostingRegressor": {"model": GradientBoostingRegressor(),
                                  "params": {"n_estimators": [50, 100, 200],
                                             "learning_rate": [0.01, 0.1, 1.0],
                                             "max_depth": [3, 5, 10]}},
    "LinearRegression": {"model": LinearRegression(),
                         "params": {}},
    "RidgeRegression": {"model": Ridge(),
                        "params": {"alpha": [0.01, 0.1, 1.0, 10.0]}},
    "LassoRegression": {"model": Lasso(),
                        "params": {"alpha": [0.01, 0.1, 1.0, 10.0]}},
    "ElasticNetRegression": {"model": ElasticNet(),
                             "params": {"alpha": [0.01, 0.1, 1.0, 10.0],
                                        "l1_ratio": [0.25, 0.5, 0.75]}},
    "DecisionTreeRegressor": {"model": DecisionTreeRegressor(),
                              "params": {"max_depth": [3, 5, 10]}}
}

#
for model_name, config in models_config.items():
    model = GridSearchCV(config['model'], config['params'], cv=3)
    model.fit(p_train, q_train)
    q_pred = model.predict(p_test)
    mse = mean_squared_error(q_test, q_pred)
    r2 = r2_score(q_test, q_pred)
    print(f"Model: {model_name}")
    print(f"Best Parameters: {model.best_params_}")
    print(f"MSE: {mse:.3f}")
    print(f"R^2: {r2:.3f}")
    print(f"Predictions: {q_pred[:5]}")
    print("=" * 50)

rf_params = {'n_estimators': [10, 15, 20], 'max_depth': [20, 23, 25], 'min_samples_leaf': [5, 7, 10]}
rf_gs = GridSearchCV(estimator=RandomForestRegressor(), param_grid=rf_params)
rf_gs.fit(p_train, q_train)
rf_best_model = rf_gs.best_estimator_
rf_pred = rf_best_model.predict(p_test)
rf_mse = mean_squared_error(q_test, rf_pred)
print("RandomForestRegressor - Best Params:", rf_gs.best_params_)
print("RandomForestRegressor - MSE:", rf_mse)

gb_params = {'n_estimators': [67, 89], 'min_samples_leaf': [1, 2], 'max_depth': [5, 7], 'min_impurity_decrease': [0.1, 0.5]}
gb_gs = GridSearchCV(estimator=GradientBoostingRegressor(), param_grid=gb_params)
gb_gs.fit(p_train, q_train)
gb_best_model = gb_gs.best_estimator_
gb_pred = gb_best_model.predict(p_test)
gb_mse = mean_squared_error(q_test, gb_pred)
print("GradientBoostingRegressor - Best Params:", gb_gs.best_params_)
print("GradientBoostingRegressor - MSE:", gb_mse)

lin_reg = LinearRegression()
lin_reg.fit(p_train, q_train)
lin_pred = lin_reg.predict(p_test)
lin_mse = mean_squared_error(q_test, lin_pred)
print("LinearRegression - MSE:", lin_mse)

ridge_params = {'alpha': [0.01, 0.1, 1.0, 10.0], 'max_iter': [30, 40, 50]}
ridge_gs = GridSearchCV(Ridge(), param_grid=ridge_params)
ridge_gs.fit(p_train, q_train)
ridge_best_model = ridge_gs.best_estimator_
ridge_pred = ridge_best_model.predict(p_test)
ridge_mse = mean_squared_error(q_test, ridge_pred)
print("RidgeRegression - Best Params:", ridge_gs.best_params_)
print("RidgeRegression - MSE:", ridge_mse)

lasso_params = {'alpha': [0.01, 0.1, 1.0, 10.0], 'max_iter': [30, 40, 50]}
lasso_gs = GridSearchCV(Lasso(), param_grid=lasso_params)
lasso_gs.fit(p_train, q_train)
lasso_best_model = lasso_gs.best_estimator_
lasso_pred = lasso_best_model.predict(p_test)
lasso_mse = mean_squared_error(q_test, lasso_pred)
print("LassoRegression - Best Params:", lasso_gs.best_params_)
print("LassoRegression - MSE:", lasso_mse)

elastic_net_params = {'alpha': [0.01, 0.1, 1.0, 10.0], 'l1_ratio': [0.25, 0.5, 0.75], 'max_iter': [30, 40, 50]}
elastic_net_gs = GridSearchCV(ElasticNet(), param_grid=elastic_net_params)
elastic_net_gs.fit(p_train, q_train)
elastic_net_best_model = elastic_net_gs.best_estimator_
elastic_net_pred = elastic_net_best_model.predict(p_test)
elastic_net_mse = mean_squared_error(q_test, elastic_net_pred)
print("ElasticNetRegression - Best Params:", elastic_net_gs.best_params_)
print("ElasticNetRegression - MSE:", elastic_net_mse)

dec_tree_params = {'max_depth': [4, 5, 6, 7], 'min_samples_leaf': [12, 6], 'splitter': ['best', 'random']}
dec_tree_gs = GridSearchCV(DecisionTreeRegressor(), param_grid=dec_tree_params)
dec_tree_gs.fit(p_train, q_train)
dec_tree_best_model = dec_tree_gs.best_estimator_
dec_tree_pred = dec_tree_best_model.predict(p_test)
dec_tree_mse = mean_squared_error(q_test, dec_tree_pred)
print("DecisionTreeRegressor - Best Params:", dec_tree_gs.best_params_)
print("DecisionTreeRegressor - MSE:", dec_tree_mse)

models = [
    LinearRegression(), Ridge(alpha=0.1), Lasso(alpha=0.1), ElasticNet(alpha=0.1),
    RandomForestRegressor(n_estimators=100), XGBRegressor(n_estimators=100),
    LGBMRegressor(n_estimators=100)
]

rmse_list, mae_list = [], []

for model in models:
    model.fit(p_train, q_train)
    q_pred = model.predict(p_test)
    rmse = np.sqrt(mean_squared_error(q_test, q_pred))
    mae = np.mean(np.abs(q_test - q_pred))
    rmse_list.append(rmse)
    mae_list.append(mae)

for i, model in enumerate(models):
    print(f"Model: {model.__class__.__name__}")
    print(f"RMSE: {rmse_list[i]}")
    print(f"MAE: {mae_list[i]}")
    print("="*30)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Model: RandomForestRegressor
Best Parameters: {'max_features': 'sqrt', 'n_estimators': 100}
MSE: 0.043
R^2: 0.932
Predictions: [1.445 0.426 1.876 1.428 1.726]
Model: GradientBoostingRegressor
Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}
MSE: 0.044
R^2: 0.931
Predictions: [1.429886   0.37447392 1.77794584 1.429886   1.64508835]
Model: LinearRegression
Best Parameters: {}
MSE: 0.046
R^2: 0.927
Predictions: [1.57270267 0.30069703 2.32427051 1.51344473 1.45986481]
Model: RidgeRegression
Best Parameters: {'alpha': 0.01}
MSE: 0.046
R^2: 0.927
Predictions: [1.5727064  0.30083065 2.32455366 1.51340876 1.46016776]
Model: LassoRegression
Best Parameters: {'alpha': 0.01}
MSE: 0.045
R^2: 0.930
Predictions: [1.57776873 0.34387996 2.44390553 1.50185251 1.5822758 ]
Model: ElasticNetRegression
Best Parameters: {'alpha': 0.01, 'l1_ratio': 0.25}
MSE: 0.045
R^2: 0.929
Predictions: [1.57396537 0.32157412 2.37346864 1.50782829 1.51152636]
Model: DecisionTreeRegressor
Best Par

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GradientBoostingRegressor - Best Params: {'max_depth': 7, 'min_impurity_decrease': 0.1, 'min_samples_leaf': 2, 'n_estimators': 89}
GradientBoostingRegressor - MSE: 0.04381416499090177
LinearRegression - MSE: 0.04643810173004898
RidgeRegression - Best Params: {'alpha': 0.01, 'max_iter': 30}
RidgeRegression - MSE: 0.04642495646708417


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.266e-02, tolerance: 5.359e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.016e-02, tolerance: 5.470e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.004e-03, tolerance: 5.267e

LassoRegression - Best Params: {'alpha': 0.01, 'max_iter': 50}
LassoRegression - MSE: 0.04472446428652983


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.541e-02, tolerance: 5.359e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.708e-02, tolerance: 5.470e-03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.995e-02, tolerance: 5.267e

ElasticNetRegression - Best Params: {'alpha': 0.01, 'l1_ratio': 0.25, 'max_iter': 50}
ElasticNetRegression - MSE: 0.04485231491145914
DecisionTreeRegressor - Best Params: {'max_depth': 4, 'min_samples_leaf': 6, 'splitter': 'best'}
DecisionTreeRegressor - MSE: 0.045571049445601045
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000579 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 73
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 3
[LightGBM] [Info] Start training from score 1.182500
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG